# Desafio 2: Licitações publicadas hoje no Diário Oficial da União

Neste desafio você irá **acessar o Portal de Compras do Governo Federal (ComprasNET)** e raspar a **descrição** de todas as licitações que foram publicadas hoje no Diário Oficial da União.

Antes de começar o desafio, [acesse o site](http://comprasnet.gov.br/ConsultaLicitacoes/ConsLicitacaoDia.asp) e se familiarize com ele. É esperado que tenha bastantes licitações sendo publicadas diariamente. É assim mesmo :)

Este desafio é mais livre, assim você pode experimentar mais :)

## Importando as bibliotecas

O fluxo de raspagem deve ser:

1. Acessar a primeira página que apresenta as licitações de hoje
2. Raspar a descrição de todas as licitações da página individualmente  
   2.1. Raspe como se fosse um bloco de texto corrido mesmo  
   2.2. Sem raspar nenhuma etiqueta HTML junto com o texto   
3. Adicionar os itens da página em um `DataFrame` da `pandas` apenas com os campos `data` (dia de hoje) e `descricao` (descrição da licitação)
4. Repetir os passos 1, 2 e 3 para cada página
5. Salvar o `DataFrame` em JSON

Realize aqui os `import` necessários para esta tarefa:

In [1]:
import datetime

import pandas as pd
from requests_html import HTMLSession

## Acessando a página inicial

In [2]:
url = "http://comprasnet.gov.br/ConsultaLicitacoes/ConsLicitacaoDia.asp"
session = HTMLSession()
response = session.get(url)

## Extraindo os itens da página inicial

In [3]:
licitacoes = []
rows = response.html.xpath("//form")
for row in rows:
    descricao = row.xpath("./form/table/tr[2]//text()")
    descricao = '\n'.join(descricao).strip()
    item = {
        "data": datetime.date.today(),
        "descricao": descricao,
    }
    licitacoes.append(item)

## Adicionando ao DataFrame

In [5]:
df_licitacoes = pd.DataFrame(licitacoes)

In [6]:
df_licitacoes

,data,descricao
0,2021-09-24,MINISTÉRIO DA EDUCAÇÃO\nUNIVERSIDADE FEDERAL D...
1,2021-09-24,MINISTÉRIO DA EDUCAÇÃO\nUniversidade Federal d...
2,2021-09-24,MINISTÉRIO DA EDUCAÇÃO\nINSTITUTO FEDERAL DE E...
3,2021-09-24,MINISTÉRIO DA DEFESA\nComando do Exército\nCom...
4,2021-09-24,"MINISTÉRIO DA CIÊNCIA, TECNOLOGIA, INOVAÇÃO E ..."
5,2021-09-24,PREFEITURA MUNICIPAL DE BELÉM\nCódigo da UASG:...
6,2021-09-24,Fundação Pública Estadual Hospital de Clínicas...
7,2021-09-24,PREFEITURA MUNICIPAL DE JABOATÃO DOS GUARARAPE...
8,2021-09-24,Empresa Brasileira de Serviços Hospitalares - ...
9,2021-09-24,HOSPITAL METROPOLITANO ODILON BEHERENS\nCódigo...


## Replicando para todas as páginas

Recomenda-se transformar a extração de conteúdo da página em uma função que pode ser chamada para cada página.

In [8]:
def extract_licitacoes(response):
    """Se desejar, preencha esta função"""
    licitacoes = []
    rows = response.html.xpath("//form")
    for row in rows:
        descricao = row.xpath("./form/table/tr[2]//text()")
        descricao = '\n'.join(descricao).strip()
        item = {
            "data": datetime.date.today(),
            "descricao": descricao,
        }
        licitacoes.append(item)
    return licitacoes

E continuar o fluxo de extrações, página a página.

In [10]:
page = 1
next_page_button = response.html.xpath("//input[@name='proxima']")
while next_page_button:
    page += 1
    next_page_url = f"{url}?pagina={page}"
    response = session.get(next_page_url)
    licitacoes = extract_licitacoes(response)
    df_licitacoes = df_licitacoes.append(pd.DataFrame(licitacoes))
    next_page_button = response.html.xpath("//input[@name='proxima']")

## Salvando em JSON

In [11]:
df_licitacoes.to_json('licitacoes.json', orient="records", date_format="iso")